In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np

# Load the data from the CSV file
data = pd.read_csv("okuzeni.csv")
municipalities = [ "ljubljana", "maribor", "kranj", "koper", "celje", "novo_mesto", "velenje", "nova_gorica", "krško", "ptuj", "murska_sobota", "slovenj_gradec"]

# Prepare the input and output data
N = 14  # Number of previous days to consider
M = 7   # Number of future days to predict

# Extract the relevant columns for the input data
input_data = data[municipalities].values
input_data = torch.tensor(input_data, dtype=torch.float32)

# Extract the columns for the output data (next M days)
output_data = data[municipalities].shift(-M).values
output_data = torch.tensor(output_data, dtype=torch.float32)

# Prepare the neighborhood graph
neighbors_data = pd.read_csv("sosedi.csv", header=None, sep=" ")
neighbors_graph = torch.zeros((len(municipalities), len(municipalities)), dtype=torch.float32)

for i, row in neighbors_data.iterrows():
    municipality1 = row[0]
    municipality2 = row[1]
    if municipality1 not in municipalities:
        continue
    index1 = municipalities.index(municipality1)
    index2 = municipalities.index(municipality2)
    neighbors_graph[index1][index2] = 1
    neighbors_graph[index2][index1] = 1

# Define the model
class InfectedPredictionModel(nn.Module):
    def __init__(self, num_municipalities):
        super(InfectedPredictionModel, self).__init__()
        self.num_municipalities = num_municipalities
        self.hidden_dim = 64

        self.lstm = nn.LSTM(input_size=num_municipalities, hidden_size=self.hidden_dim)
        self.fc = nn.Linear(self.hidden_dim, num_municipalities)

    def forward(self, x):
        x = x.unsqueeze(0)
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out.squeeze(0)
        output = self.fc(lstm_out)
        return output

# Create the model instance
model = InfectedPredictionModel(len(municipalities))

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 1000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = model(input_data)
    loss = criterion(output, output_data)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

# Test the model on new data
test_input = input_data[-N:]
with torch.no_grad():
    model.eval()
    predicted_output = model(test_input)

# Print the predicted number of infected people for the next M days
predicted_output = predicted_output.squeeze().numpy()
print("Predicted Number of Infected People for the Next M Days:")
for i, municipality in enumerate(municipalities):
    print(f"{municipality}: {predicted_output[i]}")


Epoch 100, Loss: nan
Epoch 200, Loss: nan
Epoch 300, Loss: nan
Epoch 400, Loss: nan
Epoch 500, Loss: nan
Epoch 600, Loss: nan
Epoch 700, Loss: nan
Epoch 800, Loss: nan
Epoch 900, Loss: nan
Epoch 1000, Loss: nan
Predicted Number of Infected People for the Next M Days:
ljubljana: [nan nan nan nan nan nan nan nan nan nan nan nan]
maribor: [nan nan nan nan nan nan nan nan nan nan nan nan]
kranj: [nan nan nan nan nan nan nan nan nan nan nan nan]
koper: [nan nan nan nan nan nan nan nan nan nan nan nan]
celje: [nan nan nan nan nan nan nan nan nan nan nan nan]
novo_mesto: [nan nan nan nan nan nan nan nan nan nan nan nan]
velenje: [nan nan nan nan nan nan nan nan nan nan nan nan]
nova_gorica: [nan nan nan nan nan nan nan nan nan nan nan nan]
krško: [nan nan nan nan nan nan nan nan nan nan nan nan]
ptuj: [nan nan nan nan nan nan nan nan nan nan nan nan]
murska_sobota: [nan nan nan nan nan nan nan nan nan nan nan nan]
slovenj_gradec: [nan nan nan nan nan nan nan nan nan nan nan nan]
